<a href="https://colab.research.google.com/github/Kulikov17/MLDS_ML/blob/main/KNN_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Метрические алгоритмы. Практика

В этом домашнем задании вы будете решать задачу классификации бутылок вина по различным характеристикам.

## Импорт библиотек, установка константных значений

In [7]:
import pandas as pd
import numpy as np

In [8]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.75

In [9]:
rng = np.random.default_rng(RANDOM_STATE)

## Загрузка данных

In [10]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

X = data.data
y = data.target

## Задание 1

Посмотрите на количество классов и количество объектов каждого класса в датасете.

**Вопрос**:  
Сколько классов в задаче?

In [11]:
y.value_counts()

1    71
0    59
2    48
Name: target, dtype: int64

## Задание 2

Мы имеем дело с многоклассовой классификацией. Кроме того, классы не очень хорошо сбалансированы, поэтому для оценки качества модели метрика *accuracy* не подойдет.

Разбейте данные на тренировочную и тестовую части:  
тестовая часть - 25% от всех данных, зафиксируйте `random_state = RANDOM_STATE`.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_SIZE, random_state=RANDOM_STATE)

**Вопрос:**

Все ли признаки в данных одного масштаба?  
Проверьте это, выведя основные числовые характеристики матрицы `X_train` методом `describe` из библиотеки `pandas`.

По полученной таблице числовых характеристик определите, какой признак измеряется в сотнях?  
(если вариантов несколько, выберите признак с наибольшим средним значением). - magnesium и proline

In [13]:
X_train.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000
mean,12.972857,2.386842,2.362180,19.433835,100.759398,2.277068,2.021203,0.363534,1.608647,5.017594,0.959444,2.610000,742.992481
std,0.829993,1.098905,0.280606,3.467312,14.999571,0.645696,1.005537,0.126923,0.576964,2.202516,0.234545,0.729961,306.867593
min,11.030000,0.890000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.420000,1.740000,0.480000,1.270000,278.000000
25%,12.290000,1.640000,2.210000,17.200000,88.000000,1.700000,1.200000,0.260000,1.250000,3.250000,0.780000,1.830000,500.000000
50%,12.990000,1.900000,2.360000,19.400000,98.000000,2.230000,2.140000,0.340000,1.560000,4.800000,0.970000,2.810000,675.000000
75%,13.690000,3.170000,2.540000,21.500000,108.000000,2.800000,2.880000,0.450000,1.960000,6.130000,1.120000,3.200000,970.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,10.800000,1.710000,4.000000,1547.000000


## Задание 3

KNN требует того, чтобы все признаки были одного масштаба, поэтому масштабируйте данные при помощи `StandardScaler`.

Напоминаем, что обучать метод нужно только по тренировочным данным, а применять и к трейну, и к тесту.

После применения `StandardScaler` преобразуйте `X_train` и `X_test` к типу `pd.DataFrame`, названия новых объектов оставьте `X_train` и `X_test`.

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_train = pd.DataFrame(data=X_train_norm, columns=X_train.columns, index=X_train.index)

X_test_norm = scaler.transform(X_test)
X_test = pd.DataFrame(data=X_test_norm, columns=X_test.columns, index=X_test.index)

Обучите KNN с параметрами по умолчанию на тренировочных данных и сделайте предсказание на тесте.

In [15]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()

clf.fit(X_train, y_train)
prediction = clf.predict(X_test)

Будем измерять качество модели по метрике weighted $f1$-score.

Чтобы выбрать тип усреднения (micro, macro, weighted) в функции `f1_score` необходимо задать этот тип в гиперпараметре `average`.

Вычислите $f1$-score на тестовых данных.

**Вопрос:**

Чему равен $f1$-score на тестовых данных?

In [16]:
from sklearn.metrics import f1_score

f1_score(y_test, prediction, average='weighted')

0.9550512333965844

## Задание 4

Попробуем улучшить модель.

Подберите оптимальное количество соседей (`n_neigbors`) из диапазона *от 3 до 30 с шагом 2* и веса соседей (`weights`):  
`uniform`, `distance` по кросс-валидации с тремя фолдами на тренировочных данных.

Используйте `GridSearchCV` и метрику `f1_weighted`.

In [17]:
from sklearn.model_selection import GridSearchCV

k_range = list(range(3, 30, 2))
weghts_param = ['uniform', 'distance']
param_grid = dict(n_neighbors=k_range, weights=weghts_param)
  
# defining parameter range
grid = GridSearchCV(clf, param_grid, cv=3, scoring='f1_weighted')
  
# fitting the model for grid search
grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23,
                                         25, 27, 29],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted')

Заново обучите KNN с наилучшими по `GridSearch` гиперпараметрами на тренировочных данных,  
сделайте предсказание на тесте и вычислите метрику `f1_weighted`.

In [18]:
grid.best_params_

{'n_neighbors': 19, 'weights': 'distance'}

In [19]:
clf_best = KNeighborsClassifier(n_neighbors=19, weights='distance')

clf_best.fit(X_train, y_train)
prediction = clf_best.predict(X_test)
f1_score(y_test, prediction, average='weighted')

0.9550512333965844

**Вопрос:**

Удалось ли при помощи подбора гиперпараметров улучшить качество модели на тестовых данных?

нет

## Задание 5

Выведите на экран матрицу ошибок.

Используйте модель с подобранными при помощи `GridSearch` гиперпараметрами.


**Вопрос:**  
По этой матрице определите, какие классы между собой путает модель?

In [20]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, prediction)

array([[15,  0,  0],
       [ 1, 16,  1],
       [ 0,  0, 12]])

0 и 1, 1 и 2 

## Бонус (эксперименты с LSH)

Скопируйте все функции из [ноутбука в уроке "Быстрый поиск соседей"](https://colab.research.google.com/drive/181MMOcTnzdMVzJr0pWzqtEG0-BV9BIHH).

In [21]:
def knn_search(query, data, k=5):

    dists = np.sqrt(np.sum((data - query) ** 2, axis=1))  # вычисляем расстояния от объекта query до всех точек датасета
    inds = np.argsort(dists)  # сортируем по возрастанию расстояний
    inds_k = inds[:k]         # берем top-k точек с наименьшими расстояниями
    
    return data[inds_k], dists[inds_k]

In [22]:
def generate_hyperplanes(data, bucket_size=16):
    m = data.shape[0]            # число объектов
    n = data.shape[1]            # число признаков
    b = m // bucket_size         # количество корзин
    h = int(np.log2(b))          # количество гиперплоскостей
    H = rng.normal(size=(h, n))  # гиперплоскости, заданные своими нормалями
    return H

In [23]:
def hamming_hash(data, hyperplanes):
    b = len(hyperplanes)
    hash_key = (data @ hyperplanes.T) >= 0
    
    dec_vals = np.array([2 ** i for i in range(b)], dtype=int)
    hash_key = hash_key @ dec_vals
    
    return hash_key

In [24]:
def locality_sensitive_hash(data, hyperplanes):
    hash_vals = hamming_hash(data, hyperplanes)
    hash_table = {}
    for i, v in enumerate(hash_vals):
        if v not in hash_table:
            hash_table[v] = set()
        hash_table[v].add(i)
        
    return hash_table

In [25]:
def approx_knn_search(query, data, k=5, bucket_size=16):
    candidates = set()

    hyperplanes = generate_hyperplanes(data)
    hash_table = locality_sensitive_hash(data, hyperplanes) # формируем хеш-таблицу по датасету

    query_hash = hamming_hash(query, hyperplanes)
    if query_hash in hash_table:
        candidates = candidates.union(hash_table[query_hash])
    candidates = np.stack([data[i] for i in candidates], axis=0) # находим кандидатов = объекты, попадающие с query в одну корзину

    return knn_search(query, candidates, k=k) # с помощью KNN ищем ближайших соседей только из найденных кандидатов

При помощи `knn_search` найдите ближайших соседей к вину `X_test.iloc[0]` в **тренировочных** данных.

Обратите внимание, что функция `knn_search` принимает на вход `np.array`, а не `pd.DataFrame`. Поэтому переведите аргументы в `np.array`, приписав к необходимому объекту $X$: `X.values`.

In [27]:
%%time

neighbors, dists = knn_search(X_test.iloc[0].values, X_train.values)  
for i, (neighbor, dist) in enumerate(zip(neighbors, dists)):
    print(f"top {i + 1}: dist = {dist}")

top 1: dist = 1.7873320988662762
top 2: dist = 1.9809624052592747
top 3: dist = 2.1601016426727613
top 4: dist = 2.270974564253457
top 5: dist = 2.3117955237515453
CPU times: user 2.99 ms, sys: 1.01 ms, total: 3.99 ms
Wall time: 6.85 ms


Выведите на экран признаки объекта `X_test.iloc[0]` и признаки ближайшего найденного соседа.

In [28]:
X_test.iloc[0]

alcohol                         0.806832
malic_acid                      0.651425
ash                             0.707637
alcalinity_of_ash              -1.225688
magnesium                       1.019911
total_phenols                   0.657478
flavanoids                      1.007035
nonflavanoid_phenols           -1.530581
proanthocyanins                 0.089343
color_intensity                 0.037556
hue                             0.002381
od280/od315_of_diluted_wines    1.031337
proline                         0.333672
Name: 19, dtype: float64

In [29]:
neighbors[0]

array([ 1.50827494,  1.46438454,  0.52877788, -1.80468392,  1.82295821,
        1.12384935,  1.01701759, -1.2933229 ,  0.82004316,  0.03755595,
       -0.29719787,  1.26510707,  0.05563253])

**Вопрос:**

Можно ли сказать, что в тренировочных данных есть вино, почти такое же как `X_test.iloc[0]`? (все признаки почти одинаковые)

Какое расстояние между объектом запроса и первым ближайшим соседом?

Теперь найдите ближайшего соседа при помощи `approx_knn_search`.

In [38]:
%%time

neighbors, dists = approx_knn_search(X_test.iloc[0].values, X_train.values)  
for i, (neighbor, dist) in enumerate(zip(neighbors, dists)):
    print(f"top {i + 1}: dist = {dist}")

top 1: dist = 1.787332098866276
top 2: dist = 1.9809624052592747
top 3: dist = 2.270974564253457
top 4: dist = 2.3117955237515453
top 5: dist = 2.3596474358737085
CPU times: user 991 µs, sys: 1.03 ms, total: 2.02 ms
Wall time: 1.73 ms


Ближайший сосед при помощи KNN+LSH может быть найден не точно или не с первого запуска.  
Запустите последнюю ячейку несколько раз и убедитесь, что ближайший сосед находится верно за несколько запусков.

**Вопрос:**

Запустите `knn_search` и `appox_knn_search` несколько раз и сравните время запусков. Какой из подходов в этой задаче работает быстрее?